In [1]:
import random
from data_generator.csv_data_processor import CSVDataProcessor
from utils.util_funcs import get_row_count, display_df, remove_data, verify_ranking, verify_ranking_counts
from utils.read_write import read_postgres_table
from spark_instance import spark
from pyspark.sql import Window

from pyspark.sql.functions import col, lower, lit, rand, array, concat, when, row_number, concat_ws, date_format, split, size
  

/Users/jacobbickerstaff/Desktop/Development/health-data-etl-kaggle/venv/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
24/03/26 07:35:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# csv_reader = CSVDataProcessor(spark, "data/healthcare_dataset.csv")
# 
# # Read the CSV file
# raw_df = csv_reader.run()


In [3]:
# min_age_days = 1 * 365  # Minimum age in days (18 years)
# max_age_days = 90 * 365  # Maximum age in days (90 years)
# 
# raw_df = (raw_df.withColumn("DOB", expr(f"date_sub(current_date(), CAST(round(rand() * ({max_age_days} - {min_age_days}) + {min_age_days}) AS INT))"))
#         .withColumn("Age", floor(datediff(current_date(), col("DOB")) / 365.25)))




In [4]:
# above is commented out as it's saved in the database:
df = read_postgres_table("dob_age_raw_data")

In [5]:
from constants.admission_types_dataset import admission_mapping, AdmissionTypes
flattened = [
    (
        top_level.name,
        sub_level_key.name,
        list(sub_level_info.get("stay_types")),
        sub_level_info.get("tests"),  
        random.choice(sub_level_info.get("doctors"))
    )
    for top_level, sub_level_dict in admission_mapping.items()
    for sub_level_key, sub_level_info in sub_level_dict.items()
]



In [6]:
flattened

[('EMERGENCY',
  'INJURY_RTC',
  ['Inpatient', 'Day Patient'],
  ['X-rays', 'CT scans', 'MRI', 'Ultrasound', 'Blood tests'],
  'Dr. Michelle Brown'),
 ('EMERGENCY',
  'SELF_INFLICTED',
  ['Inpatient'],
  ['Psychological assessment',
   'X-rays (for physical injuries)',
   'Blood tests',
   'Toxicology screening'],
  'Dr. Kayla Gilbert'),
 ('EMERGENCY',
  'CARDIOLOGY',
  ['Inpatient', 'Day Patient'],
  ['ECG',
   'Echocardiogram',
   'Stress tests',
   'Cardiac catheterization',
   'Blood tests'],
  'Dr. Rebecca Dennis'),
 ('EMERGENCY',
  'NEUROLOGY',
  ['Inpatient', 'Day Patient', 'Outpatient'],
  ['MRI or CT scans of the brain',
   'Electroencephalogram (EEG)',
   'Lumbar puncture',
   'Nerve conduction studies',
   'Blood tests'],
  'Dr. Robin Clayton'),
 ('EMERGENCY',
  'GASTROENTEROLOGY',
  ['Inpatient', 'Day Patient', 'Outpatient'],
  ['Endoscopy',
   'Colonoscopy',
   'Blood tests',
   'Stool tests',
   'Abdominal ultrasound',
   'CT scan'],
  'Dr. Andrea Jones'),
 ('EMERGENCY',


In [7]:
from constants.condition_probabilities import condition_age_probability_dict

flattened_condition_probabilities = [
    (
        sub_admission,
        condition,
        gender if gender in ['male', 'female'] else None,
        float(age_range[0]),
        float(age_range[1]) if len(age_range) > 1 else float('inf'),
        float(probability) 
    )
    for sub_admission, conditions in condition_age_probability_dict.items()
    for condition, genders_or_age_prob_list in conditions.items()
    for gender, age_prob_list in (genders_or_age_prob_list.items() if isinstance(genders_or_age_prob_list, dict) else [(None, genders_or_age_prob_list)])
    for age_range, probability in age_prob_list
]



In [8]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType

columns = ["condition_admission_type", "condition", "gender", "age_min", "age_max", "probability"]
schema = StructType([
    StructField("condition_admission_type", StringType(), True),
    StructField("condition", StringType(), True),
    StructField("condition_gender", StringType(), True),  # Assuming gender can be specific probability for gender 'male', 'female', or null
    StructField("age_min", FloatType(), False),
    StructField("age_max", FloatType(), False),
    StructField("probability", FloatType(), False)
])



condition_prob_df = spark.createDataFrame(data=flattened_condition_probabilities, schema=schema)


In [9]:
condition_prob_df.show()

+------------------------+--------------------+----------------+-------+-------+-----------+
|condition_admission_type|           condition|condition_gender|age_min|age_max|probability|
+------------------------+--------------------+----------------+-------+-------+-----------+
|                  CANCER|      Bladder Cancer|            NULL|    0.0|   10.0|        0.0|
|                  CANCER|      Bladder Cancer|            NULL|   11.0|   17.0|     1.8E-4|
|                  CANCER|      Bladder Cancer|            NULL|   18.0|   25.0|     2.0E-4|
|                  CANCER|      Bladder Cancer|            NULL|   25.0|   34.0|    0.00224|
|                  CANCER|      Bladder Cancer|            NULL|   35.0|   44.0|    0.00448|
|                  CANCER|      Bladder Cancer|            NULL|   45.0|   59.0|    0.00673|
|                  CANCER|      Bladder Cancer|            NULL|   45.0|   54.0|    0.00897|
|                  CANCER|      Bladder Cancer|            NULL|   55.

In [10]:
df.show()

+--------------------+---+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+----------+
|                name|Age|gender|blood_type|medical_condition|date_of_admission|            doctor|            hospital|insurance_provider|    billing_amount|room_number|admission_type|discharge_date| medication|test_results|       DOB|
+--------------------+---+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+----------+
|     Tiffany Ramirez| 39|Female|        O-|         Diabetes|       2022-11-17|    Patrick Parker|    Wallace-Hamilton|          Medicare| 37490.98336352819|        146|      Elective|    2022-12-01|    Aspirin|Inconclusive|1984-09-30|
|         Ruben Burns| 16|  Male|        O+|        

In [11]:
"""
STEP 1 
    From the flattened data list we wish to create a DataFrame. This contains all the possible combinations for the given 
    top level admissions, sub level admissions, stay types and list of tests available from the admission_mapping, stay_type and admission_tests lists or dictionaries in admission_types_test_dataset.py
"""

mapping_df = spark.createDataFrame(flattened, ["top_level_admission", "sub_level_admission", "stay_types", "possible_tests", "doctor"])

display_df(mapping_df, 45)

,top_level_admission,sub_level_admission,stay_types,possible_tests,doctor
0,EMERGENCY,INJURY_RTC,"[Inpatient, Day Patient]","[X-rays, CT scans, MRI, Ultrasound, Blood tests]",Dr. Michelle Brown
1,EMERGENCY,SELF_INFLICTED,[Inpatient],"[Psychological assessment, X-rays (for physica...",Dr. Kayla Gilbert
2,EMERGENCY,CARDIOLOGY,"[Inpatient, Day Patient]","[ECG, Echocardiogram, Stress tests, Cardiac ca...",Dr. Rebecca Dennis
3,EMERGENCY,NEUROLOGY,"[Inpatient, Day Patient, Outpatient]","[MRI or CT scans of the brain, Electroencephal...",Dr. Robin Clayton
4,EMERGENCY,GASTROENTEROLOGY,"[Inpatient, Day Patient, Outpatient]","[Endoscopy, Colonoscopy, Blood tests, Stool te...",Dr. Andrea Jones
5,EMERGENCY,RESPIRATORY,"[Inpatient, Day Patient, Outpatient]","[Pulmonary function tests, Chest X-ray, CT sca...",Dr. Jared Green
6,EMERGENCY,INFECTIOUS_DISEASES,"[Inpatient, Day Patient]","[Blood cultures, PCR tests, Antibody tests, ch...",Dr. Christina Castillo
7,EMERGENCY,MATERNITY,"[Inpatient, Day Patient, Outpatient]","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Bruce Lane
8,EMERGENCY,PSYCHIATRIC,"[Inpatient, Day Patient, Outpatient]","[Psychological evaluation, Blood tests, Brain ...",Dr. Sarah Harris
9,EMERGENCY,ORTHOPEDICS,"[Inpatient, Day Patient, Outpatient]","[X-rays, MRI, CT scans, Bone scans, Blood tests]",Dr. James Anderson


In [12]:
joined_tbl = mapping_df.join(condition_prob_df, on=[mapping_df.sub_level_admission == condition_prob_df.condition_admission_type], how="left")

In [13]:
joined_tbl.show(n=2000)

+-------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+----------------+-------+--------+-----------+
|top_level_admission| sub_level_admission|          stay_types|      possible_tests|              doctor|condition_admission_type|           condition|condition_gender|age_min| age_max|probability|
+-------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+----------------+-------+--------+-----------+
|          EMERGENCY|          INJURY_RTC|[Inpatient, Day P...|[X-rays, CT scans...|  Dr. Michelle Brown|                    NULL|                NULL|            NULL|   NULL|    NULL|       NULL|
|          EMERGENCY|           NEUROLOGY|[Inpatient, Day P...|[MRI or CT scans ...|   Dr. Robin Clayton|                    NULL|                NULL|            NULL|   NULL|    NULL|       NULL|
|         

In [14]:
"""
STEP 2 
    Create a list of admission_types randomly assign this to the original patient in the original data set, 
    whilst dropping the original admission_type column. Then join with mapping_df on top_level_admission col to give access to possible 
    conditions, mappings and so on.
"""
admission_type_names = [member.name for member in AdmissionTypes]

keys_array = array([lit(name) for name in admission_type_names])

# Define constants and conditions
female_only = ["MATERNITY", 'obstetrics']
is_female = lower(col('gender')) == 'female'
is_pediatric = col("Age") < 18
is_geriatric = (col("Age") >= 65)

df = (df.withColumn("is_female", is_female)
        .withColumn("is_pediatric", is_pediatric)
        .withColumn("is_geriatric", is_geriatric)
        .withColumn("unique_id", concat_ws("_", "name", date_format("DOB", "yyyyMMdd")))
        .drop("doctor", "medical_condition", "test_results", "medication", "admission_type")
      )
get_row_count(df, True)



10000


10000

In [17]:
# joined_tbl.show()

In [18]:
from constants.admission_types_dataset import SubAdmissionTypes
from utils.conditions_creator import ConditionsCreator
"""
STEP 3  
Create Enum class df and join it on to main driver df. 
Then create conditions 
"""
enum_values = [e.name for e in SubAdmissionTypes]

enum_df = spark.createDataFrame(enum_values, StringType()).toDF("admission_type")
df_enum_cross = df.crossJoin(enum_df)
con = ConditionsCreator(spark, df_enum_cross, joined_tbl)

selected_conditions_df = con.runner()


In [19]:
selected_conditions_df.show(n=20000)

+--------------------+--------------------+---------------+--------------------------+--------------------+
|           unique_id|          stay_types|submission_type|chosen_top_level_admission|    chosen_condition|
+--------------------+--------------------+---------------+--------------------------+--------------------+
|Aaron White PhD_1...|[Inpatient, Day P...|     CARDIOLOGY|         HOSPITAL_REFERRAL|Hypertension (Hig...|
|Abigail Mack_2020...|        [Outpatient]|    DERMATOLOGY|             SELF_REFERRAL|  pediatric_Impetigo|
|Abigail Young_198...|[Inpatient, Day P...|     CARDIOLOGY|                 EMERGENCY|         Arrhythmias|
|Adam Taylor_20050811|        [Outpatient]|    DERMATOLOGY|             SELF_REFERRAL|            Impetigo|
|Adrian Mooney_201...|        [Outpatient]|    DERMATOLOGY|             SELF_REFERRAL|pediatric_Hives (...|
|Adrian Valencia_2...|        [Outpatient]|    DERMATOLOGY|             SELF_REFERRAL|pediatric_Hives (...|
|Adriana Ochoa_196...|[Inpat

In [ ]:
"""
STEP 4
    Create a row ranking using a unique cols, created from stay_name and unique_id. 
    Verification done below. 
"""
# 
# df =  df.withColumn('stay_name', 
#                    when(col('stay_type') == 'out_patient', concat(col('name'), lit('_out_patient')))
#                    .when(col('stay_type') == 'inpatient', concat(col('name'), lit('_inpatient')))
#                    .when(col('stay_type') == 'day_patient', concat(col('name'), lit('_day_patient')))
#                    .otherwise(col('name'))
#                   )
# 
# 
# # Define a window specification that partitions data by 'top_level_admission' (or another unique patient identifier if needed)
# windowSpec = Window.partitionBy('stay_name', 'unique_id').orderBy(rand())
# 
# # Assign row numbers within each partition in a random order
# ranked_df = df.withColumn("row_num", row_number().over(windowSpec))


In [19]:

split_col = split(df['name'], ' ')

df_renamed = (df.withColumn('name', 
                    when(size(split_col) == 4, concat(split_col.getItem(1).substr(1, 1), lit('. '), split_col.getItem(2).substr(1, 1), lit('. '), split_col.getItem(3)))
                    .when(size(split_col) == 3, concat(split_col.getItem(1).substr(1, 1), lit('. '), split_col.getItem(2)))
                    .otherwise(
                        concat(split_col.getItem(0).substr(1, 1), lit('. '), split_col.getItem(1))
                               )
                    )
      )


In [20]:
df_renamed.show()

+------------+---+------+----------+-----------------+--------------------+------------------+------------------+-----------+--------------+----------+---------+------------+------------+--------------------+
|        name|Age|gender|blood_type|date_of_admission|            hospital|insurance_provider|    billing_amount|room_number|discharge_date|       DOB|is_female|is_pediatric|is_geriatric|           unique_id|
+------------+---+------+----------+-----------------+--------------------+------------------+------------------+-----------+--------------+----------+---------+------------+------------+--------------------+
|  T. Ramirez| 39|Female|        O-|       2022-11-17|    Wallace-Hamilton|          Medicare| 37490.98336352819|        146|    2022-12-01|1984-09-30|     true|       false|       false|Tiffany Ramirez_1...|
|    R. Burns| 16|  Male|        O+|       2023-06-01|Burke, Griffin an...|  UnitedHealthcare| 47304.06484547511|        404|    2023-06-15|2007-04-05|    false|   

In [21]:
(df_renamed.select("name", "DOB", "Age", "gender", "blood_type", "date_of_admission", "discharge_date", "top_level_admission", "sub_level_admission", "possible_tests", "conditions", "doctor", "hospital", "room_number", "insurance_provider", "billing_amount", "stay_type", "is_female", "is_geriatric", "is_pediatric", "stay_name", "row_num", "unique_id")
 .show(n=200))

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `top_level_admission` cannot be resolved. Did you mean one of the following? [`date_of_admission`, `billing_amount`, `is_geriatric`, `is_pediatric`, `is_female`].;
'Project [name#802, DOB#15, Age#1L, gender#2, blood_type#3, date_of_admission#5, discharge_date#12, 'top_level_admission, 'sub_level_admission, 'possible_tests, 'conditions, 'doctor, hospital#7, room_number#10, insurance_provider#8, billing_amount#9, 'stay_type, is_female#211, is_geriatric#248, is_pediatric#229, 'stay_name, 'row_num, unique_id#268]
+- Project [CASE WHEN (size(split(name#0,  , -1), true) = 4) THEN concat(substring(split(name#0,  , -1)[1], 1, 1), . , substring(split(name#0,  , -1)[2], 1, 1), . , split(name#0,  , -1)[3]) WHEN (size(split(name#0,  , -1), true) = 3) THEN concat(substring(split(name#0,  , -1)[1], 1, 1), . , split(name#0,  , -1)[2]) ELSE concat(substring(split(name#0,  , -1)[0], 1, 1), . , split(name#0, 

In [ ]:
df_new = df_renamed.drop("row_num")

windowSpec = Window.partitionBy('stay_name', 'unique_id').orderBy(rand())

# Assign row numbers within each partition in a random order
df_new_part = df_new.withColumn("row_num", row_number().over(windowSpec))

df_new_part.select(["name", "DOB", "Age", "gender", "blood_type", "date_of_admission", "discharge_date", "top_level_admission", "sub_level_admission", "possible_tests", "conditions", "doctor", "hospital", "room_number", "insurance_provider", "billing_amount", "stay_type", "is_female", "is_geriatric", "is_pediatric", "stay_name", "row_num", "unique_id"]).where(col("stay_name") =="Tiffany Ramirez").show() 

In [ ]:
get_row_count(df_new_part)

In [ ]:
["name", "DOB", "Age", "gender", "blood_type", "date_of_admission", "discharge_date", "top_level_admission", "sub_level_admission", "possible_tests", "conditions", "doctor", "hospital", "room_number", "insurance_provider", "billing_amount", "stay_type", "is_female", "is_geriatric" "is_pediatric", "stay_name", "row_num", "unique_id"]

In [ ]:
df_new_part.show(n=2000)

In [ ]:
# TODO:  
# medical condition to be  chosen 
# tests to be chosen
# admission date to be checked again dob,  
# TODO filter on is pediatric, geriatric and is_female to be done here and same people with dob? needs  to be considered 
# drop stay_name and unique_id 


In [ ]:
df = join_with_condition_prob_df.select([col(c).cast(StringType()).alias(c) for c in join_with_condition_prob_df.columns])
df.show()
df.repartition(10).write.csv('./temp_data/join_with_condition_prob_df/renamed.csv', mode = 'overwrite', header=True)

In [ ]:
spark.stop()